In [ ]:
DATASETS_DIR = "C:/Users/mgonz/Desktop/yelp/"
# DATASETS_DIR = "./"
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df_train = pd.read_csv(DATASETS_DIR + "train_reviews.csv")
df_test = pd.read_csv(DATASETS_DIR + "test_reviews.csv")

### df_train

In [4]:
print("Train set shape:", df_train.shape)
df_train.head(5)

Train set shape: (967784, 9)


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,ZZO43qKB-s65zplC8RfJqw,-1BSu2dt_rOAqllw9ZDXtA,smkZq4G1AOm4V6p3id5sww,5.0,0,0,0,Fantastic fresh food. The greek salad is amazi...,2016-09-30 15:49:32
1,vojXOF_VOgvuKD95gCO8_Q,xpe178ng_gj5X6HgqtOing,96_c_7twb7hYRZ9HHrq01g,1.0,2,0,1,Been a patient at Largo Med/Diagnostic Clinic ...,2020-12-09 14:39:51
2,KwxdbiseRlIRNzpgvyjY0Q,axbaerf2Fk92OB4b9_peVA,e0AYjKfSF0DL-5C1CpOq6Q,4.0,0,0,0,The location is convenient to my campus so I d...,2013-09-04 16:19:51
3,3mwoBcTy-2gMh0L91uaIeA,_GOiybb0rImYKJfwyxEaGg,vF-uptiQ34pVLHJKzPHUlA,5.0,0,0,0,I agree with all the other compliments posted ...,2019-03-02 12:24:14
4,XfWf7XsBWs3kYyYq7Ns1ZQ,ojWKg3B5pH3ncAsxun3kUw,X28XK71RuEXPapeyUOwNzg,5.0,10,4,7,"Wanting to help out the local economy, I thoug...",2020-04-23 18:26:29


In [5]:
df_train["stars"].unique()

array([5., 1., 4., 2., 3.])

### df_test

In [6]:
print("Test set shape:", df_test.shape)
df_test.head(5)

Test set shape: (414765, 8)


,review_id,user_id,business_id,useful,funny,cool,text,date
0,ieYPmCImINjPzTDFmEKBKA,79F9QrQSet-b1yRCIM243Q,sXSUzImYOcRRI3xtG2M85g,1,0,1,Amazing coffee and chill atmosphere. The staff...,2018-01-29 04:33:28
1,QIkJ8fZ4yx_QaHahWWszAA,chuM6TBkFHtTwJ6z96Hj1A,Ipt9ga67vVC_2ob3YmVwNA,4,0,2,I pass by this joint every time I make a run t...,2011-01-10 03:10:49
2,seR2KhblYMWg-k9zzN6aYA,hF68a0mpu97u0oaryFYhyg,_RG4IByyBR528CMc7DefJA,2,0,0,Came here when my kitten got very sick by the ...,2015-09-06 15:29:02
3,BToo00Fi5pfJFA5MI2HM5g,G4yX5Q1tFfwSucFOmiyjdA,xxlbRiWWQkk-6LST3Hd12g,2,0,0,So I'll preface by saying we did have an overa...,2015-09-14 00:49:17
4,FHJAzi1imodBit3RWK7zQA,Srqi1xb7exdB9uRHxDeEkw,LgGqdFLD7-ca0Z9F_q4Fuw,0,0,0,This place is a joke. Worst bar service ever. ...,2015-07-24 01:03:40


### Creamos dataset de train y test del DF_TRAIN

In [7]:
X_train, X_val, y_train, y_val = train_test_split(df_train["text"], df_train["stars"], test_size=0.2, random_state=42)

In [8]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)

X_train shape: (774227,)
X_val shape: (193557,)
y_train shape: (774227,)
y_val shape: (193557,)


In [9]:
X_train.head(5)

,text
300516,"Love this place. Both times I came here, the c..."
695208,Costa Vista is outstanding and flavorful food...
921436,I'm a big fan. A friend showed me a pic of his...
397552,"Great team of PTs, the staff is friendly and k..."
953340,Do not sleep on them. If you are looking for a...


### Pasamos los textos a embeddings

In [18]:
from sentence_transformers import SentenceTransformer
model_encoder = SentenceTransformer('all-MiniLM-L6-v2')
# print("TEXTO:", X_train[1])
X_train = model_encoder.encode(X_train.tolist(), convert_to_tensor=True)
X_val = model_encoder.encode(X_val.tolist(), convert_to_tensor=True)


### Entrenamos el modelo

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
import numpy as np

# Entrenamiento del modelo
model_xgb = XGBRegressor(tree_method="hist", n_estimators=100, learning_rate=0.15, random_state=42)
model_xgb.fit(X_train.cpu().numpy(), y_train)


In [21]:
# Predicción
y_pred = model_xgb.predict(X_val.cpu().numpy())

# Métricas de regresión
rmse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print("📊 Métricas de regresión:")
print(f" - RMSE: {rmse:.4f}")
print(f" - MAE: {mae:.4f}")
print(f" - R²: {r2:.4f}")

# (Opcional) Métrica tipo 'accuracy' si las puntuaciones son enteras
if all(float(y).is_integer() for y in y_val):
    acc = accuracy_score(y_val, np.round(y_pred))
    print(f" - Accuracy (puntuaciones redondeadas): {acc:.4f}")

📊 Métricas de regresión:
 - RMSE: 0.8176
 - MAE: 0.6844
 - R²: 0.6263
 - Accuracy (puntuaciones redondeadas): 0.4788


### Exportamos CSV con los resultados

In [ ]:
# Paso 1: Extraer los textos y los IDs
texts_test = df_test["text"].tolist()
review_ids = df_test["review_id"].tolist()

# Paso 2: Sacar embeddings (usando el mismo modelo que usaste para entrenamiento)
X_test_embeddings = model_encoder.encode(texts_test, convert_to_tensor=True)

# Paso 3: Predecir puntuaciones con XGBoost
y_pred_test = model_xgb.predict(X_test_embeddings.cpu().numpy())

# Paso 4: Crear DataFrame con las predicciones
df_predictions = pd.DataFrame({
    "review_id": review_ids,
    "stars": y_pred_test
})

# # Paso 5: Redondear si lo deseas (opcional)
# df_predictions["stars"] = df_predictions["stars"].round(1)
# Paso 5: Redondear a entero y forzar que termine en .0
df_predictions["stars"] = df_predictions["stars"].round().astype(int).astype(float)


# Paso 6: Exportar a CSV
df_predictions.to_csv("predicciones_stars.csv", index=False)

print("✅ CSV generado con éxito.")


✅ CSV generado con éxito.
